# BELKA: Molecule Representations for ML Tutorial

Author:

- Stephen Lee ([LinkedIn](https://www.linkedin.com/in/stephendongsoolee/))

<a name="background"></a>

# 📖 Background

---

**SMILES (Simplified Molecular Input Line Entry System)** notation is a way of representing chemical structures as a string, allowing for easy input, storage, and manipulation of molecular data. The ability to convert these SMILES strings into meaningful numerical representations (embeddings) is crucial for applying machine learning techniques to chemical compounds.

This notebook aims to provide helpful, high-level material and demo code to generate various types of molecular embeddings that can be generated from SMILES and used for machine learning. Specifically, the following types of molecular embeddings are covered:

- [Descriptors](#descriptors)
- [Fingerprints](#fingerprints)
- [Graphs](#graph)
- [Mol2Vec Embeddings](#mol2vec)
- [Chemical Language Model Embeddings](#clm)

Please leave a comment if you have questions or suggestions!


<a name="setup"></a>

# 🛠️ Environment Setup

---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# for basic SMILES operations, descriptors and fingerprints
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors

# for graphs
import dgl
import torch

# for mol2vec
from gensim.models import word2vec
from mol2vec.features import (
    mol2alt_sentence,
    mol2sentence,
    MolSentence,
    DfVec,
    sentences2vec,
)

# for chemical language models
from transformers import AutoModel, AutoTokenizer

<a name="data"></a>

# 🗃️ Load Data

---


In [ ]:
# load sample of training data for demo
df = pd.read_parquet("../../data/raw/test.parquet")
df.head()

In [ ]:
df = df[:1000]

In [ ]:
# assign unique molecule_smiles values to variable. This is all we need for the purpose of this demo
smiles = df["molecule_smiles"].unique()
bb1 = df["buildingblock1_smiles"].unique()
bb2 = df["buildingblock2_smiles"].unique()
bb3 = df["buildingblock3_smiles"].unique()
print(len(smiles))

SMILES can be converted into standardized data structures called Mol objects which provide rich representations of molecular structures. Many of the embedding generation featured in this notebook relies on Mol objects as input.

Mol objects are generated using the RDKit package, a widely used cheminformatics library for molecular modeling, chemical analysis and computational chemistry.


In [ ]:
# obtain RDKit mol objects from SMILES
mols = pd.Series(smiles).apply(Chem.MolFromSmiles)
mols_bb1 = pd.Series(bb1).apply(Chem.MolFromSmiles)
mols_bb2 = pd.Series(bb2).apply(Chem.MolFromSmiles)
mols_bb3 = pd.Series(bb3).apply(Chem.MolFromSmiles)

In [ ]:
# mol objects can be readily viewed as 2d molecular drawings like so
mols[0]

In [ ]:
mols_bb1[0]

<a name="descriptors"></a>

# 🧪 Molecular Descriptors

---

Molecular descriptors are quantitative properties that capture information about the chemical structure that can be used effectively in cheminformatics, particularly for the modeling and analysis of chemical compounds. Descriptors can pertain to various aspects of a chemical structure, such as its constitution, topology, geometry, electron distribution, and hydrophobicity.

The following code demonstrates how to generate molecular descriptors from SMILES.

References:

- [RDKit Documentation](https://www.rdkit.org/docs/)
- [RDKit GitHub](https://github.com/rdkit/rdkit)


In [ ]:
# generate RDKit descriptors from Mol objects
def generate_descriptors(mols):
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(
        [x[0] for x in Descriptors._descList]
    )

    mol_descriptors = []
    for mol in mols:
        rdkit_descriptors = calc.CalcDescriptors(mol)
        mol_descriptors.append(rdkit_descriptors)

    desc_names = calc.GetDescriptorNames()
    return pd.DataFrame(mol_descriptors, columns=desc_names)


descriptor_df = generate_descriptors(mols)
descriptor_df.head()

Since molecular descriptors directly represent some characteristic of a chemical structure, they are readily usable for analysis. For example, you could examine the relationship between molecular weight and logP (measure of hydrophobicity):


In [ ]:
sns.scatterplot(x="MolWt", y="MolLogP", data=descriptor_df)
plt.show()

<a name="fingerprints"></a>

# 🗝️ Molecular Fingerprints

---

**Molecular fingerprints** are fixed-length numerical representations that encode molecular structures, usually as a binary vector. Each element (called a bit) of a fingerprint vector represents the presence of specific atomic or structural features within the chemical compound.

Below is example code to generate two types of commonly used molecular fingerprints:

- **MACCS Keys**: a 166-bit fingerprint based on a predefined list of molecular substructures or patterns
- **Morgan Fingerprints**: a fixed-length bit fingerprint based on hashed topological features of atoms and their bond connectivities within a specified radius. Also referred to as Extended-Connectivity Fingerprints (ECFPs)

References::

- Durant, J. L., Leland, B. A., Henry, D. R., & Nourse, J. G. (2002). Reoptimization of MDL keys for use in drug discovery. Journal of Chemical Information and Computer Sciences, 42(6), 1273-1280. [Link to Article](https://pubs.acs.org/doi/10.1021/ci010132r)
- Rogers, D., & Hahn, M. (2010). Extended-connectivity fingerprints. Journal of Chemical Information and Modeling, 50(5), 742-754. [Link to Article](https://pubs.acs.org/doi/10.1021/ci100050t)


In [ ]:
# generate MACCS Keys from RDKit Mol objects as a dataframe
maccs_list = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]

maccs_data = []
for maccs in maccs_list:
    bit_array = list(map(int, maccs.ToBitString()))
    maccs_data.append(
        bit_array[1:]
    )  # skip 1st bit as it is unused in RDKit's MACCSkeys.GenMACCSKeys implementation

maccs_df = pd.DataFrame(maccs_data)
maccs_df.head()

In [ ]:
# generate Morgan Fingerprints from RDKit Mol objects as a dataframe
mfpt_list = [
    AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024) for mol in mols
]

mfpt_df = pd.DataFrame([list(fpt) for fpt in mfpt_list])
mfpt_df.head()

Note that different values can be specified for radius and nBits when generating Morgan Fingerprints.

- **radius:** number of bond steps to include from each atom when calculating the fingerprint. A radius of 1 means that only the immediate neighbors are considered; a radius of 2 includes neighbors up to two bonds away, etc.
- **nBits:** total number of bits in the fingerprint (i.e. resolution). Larger nBits values increases feature dimensionality and typically increases feature sparsity.

**Bit Collision** refers to the scenario where different substructural features are mapped to the same bit position. This is undesirable as it reduces the fingerprint's ability to differentiate molecules. Increasing nBits increases the available feature space and thus lowers the chance of bit collisions.


<a name="graph"></a>

# ⚛️ Molecular Graph Embeddings

---

A graph is a type of data structure to represent some interconnected entitiy (e.g. social networks). Molecules can be represented as graphs with atoms as nodes and bonds as edges.

Below is example code to generate graphs from RDKit Mol objects using the Deep Graph Learning (DGL) package.

References::

- Wang, M., Zheng, D., Ye, Z., Gan, Q., Li, M., Song, X., Zhou, J., Ma, C., Yu, L., Gai, Y., Xiao, T., He, T., Karypis, G., Li, J., & Zhang, Z. (2019). Deep graph library: A graph-centric, highly-performant package for graph neural networks. [Link to Article](https://arxiv.org/abs/1909.01315)
- [Deep Graph Libarary Docs](https://docs.dgl.ai/)


In [ ]:
# generate molecular graphs from RDKit Mol objects
def mol_to_dgl_graph(mol):
    graph = dgl.DGLGraph()

    # add nodes
    num_atoms = mol.GetNumAtoms()
    graph.add_nodes(num_atoms)

    # add edges
    for bond in mol.GetBonds():
        # DGL graphs are directed so both directions are added
        graph.add_edges(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
        graph.add_edges(bond.GetEndAtomIdx(), bond.GetBeginAtomIdx())

    # add node features - e.g. atomic number
    node_feats = torch.tensor(
        [[atom.GetAtomicNum()] for atom in mol.GetAtoms()], dtype=torch.float32
    )
    graph.ndata["feat"] = node_feats

    # add edge features (optional) - e.g. bond type: single/double/triple as 1, 2, 3
    edge_feats = []
    for bond in mol.GetBonds():
        bond_type = bond.GetBondTypeAsDouble()
        edge_feats.append([bond_type])
        edge_feats.append([bond_type])  # add both directions
    graph.edata["type"] = torch.tensor(edge_feats, dtype=torch.float32)

    return graph


graphs = mols.apply(mol_to_dgl_graph)
print(graphs)

In [ ]:
# graph neural networks typically process graphs in batches for efficiency
batched_graph = dgl.batch(graphs.tolist())
batched_graph

<a name="mol2vec"></a>

# 💬 Mol2Vec Embeddings

---

Mol2Vec is a version of the widespread natural language processing (NLP) algorithm Word2vec, specifically adapted for molecular data. Mol2Vec represents molecules in a continuous vector space that captures chemical similarity in a mannner that is analagous to how Word2Vec captures semantic similarity between words. Mol2Vec treats chemical substructures (e.g. Morgan fingerprints with radius=1) as fundamental units of a 'molecular sentence', similar to how words are treated in NLP.

A Mol2Vec model pre-trained on 20 million molecules is available to generate 300 dimensional embeddings based on learned molecular vector representations. Below is example code to generate Mol2Vec embeddings

References:

- Jaeger, S., Fulle, S., & Turk, S. (2018). Mol2vec: unsupervised machine learning approach with chemical intuition. Journal of Chemical Information and Modeling, 58(1), 27-35. [Link to Article](https://pubs.acs.org/doi/10.1021/acs.jcim.7b00616)
- [Mol2Vec GitHub Repo](https://github.com/samoturk/mol2vec)


In [ ]:
# load pre-trained mol2vec model
mol2vec_url = (
    "https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl"
)
mol2vec_model = word2vec.Word2Vec.load(mol2vec_url)

In [ ]:
# generate molecular sentences
mol_sentences = mols.apply(lambda x: MolSentence(mol2alt_sentence(x, 1)))


# version of sentences2vec() compatible with gensim v4.0 (source: https://github.com/samoturk/mol2vec/issues/14)
def sentences2vec(sentences, model, unseen=None):
    """Generate vectors for each sentence (list) in a list of sentences. Vector is simply a
    sum of vectors for individual words.

    Parameters
    ----------
    sentences : list, array
        List with sentences
    model : word2vec.Word2Vec
        Gensim word2vec model
    unseen : None, str
        Keyword for unseen words. If None, those words are skipped.
        https://stats.stackexchange.com/questions/163005/how-to-set-the-dictionary-for-text-analysis-using-neural-networks/163032#163032

    Returns
    -------
    np.array
    """

    keys = set(model.wv.key_to_index)
    vec = []

    if unseen:
        unseen_vec = model.wv.get_vector(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(
                sum(
                    [
                        (
                            model.wv.get_vector(y)
                            if y in set(sentence) & keys
                            else unseen_vec
                        )
                        for y in sentence
                    ]
                )
            )
        else:
            vec.append(
                sum(
                    [
                        model.wv.get_vector(y)
                        for y in sentence
                        if y in set(sentence) & keys
                    ]
                )
            )
    return np.array(vec)


# generate vector embeddings from molecular sentences
mol2vec_embeddings = np.array(
    [DfVec(x) for x in sentences2vec(mol_sentences, mol2vec_model, unseen="UNK")]
)
mol2vec_embeddings[:5]

<a name="clm"></a>

# 🤖 Chemical Language Model Embeddings

---

Just as traditional language models use self-attention mechanisms to compute the representation of each language element (e.g. word in a sentence) to every other element, chemical language models use the same principle in which elements are some chemical unit (e.g. atoms) instead of words.

Below is example code to obtain learned transformer-based embeddings from two chemical language models:

- ChemBERTa: adapted for chemical SMILES from the RoBERTa architecture, trained on a dataset of 77 million molecules
- MoLFormer: another transformer-based model adapted for SMILES but trained on a larger dataset (1.1 billion molecules!)

References:

- Chithrananda, S., Grand, G., & Ramsundar, B. (2020). ChemBERTa: Large-Scale Self-Supervised Pretraining for Molecular Property Prediction. [Link to Article](https://arxiv.org/abs/2010.09885)
- Chithrananda, S., Grand, G., & Ramsundar, B. (2022). ChemBERTa-2: Towards Chemical Foundation Models. [Link to Article](https://arxiv.org/abs/2209.01712)
- Ross, J., Belgodere, B., Chenthamarakshan, V., et al. (2022). Large-scale chemical language representations capture molecular structure and properties. Nature Machine Intelligence, 4, 1256-1264. [Link to Article](https://www.nature.com/articles/s42256-022-00580-7)
- [ChemBERTa on HuggingFace Model Repo](https://huggingface.co/seyonec/ChemBERTa-zinc-base-v1)
- [MolFormer on HuggingFace Model Repo](https://huggingface.co/ibm/MoLFormer-XL-both-10pct)
- [MolFormer GitHub repo](https://github.com/IBM/molformer)


In [ ]:
# load pre-trained ChemBERTa model checkpoint and tokenizer
cb_tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MLM")
cb_model = AutoModel.from_pretrained("DeepChem/ChemBERTa-10M-MLM")
cb_model.eval()

# tokenize SMILES
cb_encoded_inputs = cb_tokenizer(
    list(smiles), padding=True, truncation=True, return_tensors="pt"
)

# calculate embeddings
with torch.no_grad():
    outputs = cb_model(**cb_encoded_inputs)

# extract pooled output
cb_embeddings = outputs.pooler_output

cb_embeddings_df = pd.DataFrame(cb_embeddings.numpy())
cb_embeddings_df.head()

In [ ]:
# load pre-trained MolFormer model checkpoint and tokenizer
mf_tokenizer = AutoTokenizer.from_pretrained(
    "ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True
)
mf_model = AutoModel.from_pretrained(
    "ibm/MoLFormer-XL-both-10pct", trust_remote_code=True
)

# tokenize SMILES
mf_encoded_inputs = mf_tokenizer(list(smiles), padding=True, return_tensors="pt")

# calculate embeddings
with torch.no_grad():
    outputs = mf_model(**mf_encoded_inputs)

# extract embeddings
mf_embeddings = outputs.pooler_output

mf_embeddings_df = pd.DataFrame(mf_embeddings.numpy())
mf_embeddings_df.head()